In [36]:
import numpy as np
from random import shuffle
import tensorflow as tf

Generating training input data. Esto viene de un ejemplo donde a partir de sequencias de 20 elementos binarios (0/1), determinar el numero de 1s (as a 21vector, with a 1 in the position of the number). Train input tiene todas las psibles permutaciones de 0/1 con 20 digitos (2^20).

In [37]:
train_input = ['{0:020b}'.format(i) for i in range(2**20)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

In [38]:
np.array(train_input).shape

(1048576, 20, 1)

Train Output

In [39]:
train_output = []
 
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    train_output.append(temp_list)

In [40]:
np.array(train_output).shape

(1048576, 21)

In [41]:
np.array(train_output)[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Generating Test Data

In [42]:
NUM_EXAMPLES = 10000
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:] #everything beyond 10,000
 
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES] #till 10,000

Design the model

In [44]:
tf.reset_default_graph()
data = tf.placeholder(tf.float32, [None, 20,1])
target = tf.placeholder(tf.float32, [None, 21])
num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

In [45]:
print(val)
print(state)

Tensor("RNN/transpose:0", shape=(?, 20, 24), dtype=float32)
LSTMStateTuple(c=<tf.Tensor 'RNN/while/Exit_2:0' shape=(?, 24) dtype=float32>, h=<tf.Tensor 'RNN/while/Exit_3:0' shape=(?, 24) dtype=float32>)


In [46]:
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)
print(val)
print(last)

Tensor("transpose_1:0", shape=(20, ?, 24), dtype=float32)
Tensor("Gather:0", shape=(?, 24), dtype=float32)


In [49]:
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
cross_entropy = -tf.reduce_sum(target * tf.log(prediction))
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

/Users/alfonso/anaconda/lib/python3.5/site-packages/tensorflow/python/ops/gradients.py:89: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Calculatin error on test data

In [17]:
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

Execution of the graph

In [18]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

In [23]:
batch_size = 1000
no_of_batches = int(len(train_input)/batch_size)
epoch = 50
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp, target: out})
    #print ("Epoch - ",str(i))
incorrect = sess.run(error,{data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
sess.close()

Epoch 50 error 18.9%
